In [ ]:
import numpy as np
import pandas as pd 
from numpy.fft import fft
from scipy.linalg import hadamard
import plotly.graph_objects as go
import time
from plotly.subplots import make_subplots

from algo import *

*Algorithms*

#### *Benchmarks*

In [13]:
sizes = np.arange(100,1_000+100,100)
delta_dense = 1.0
delta_sparse = 0.01
r = 20
rOV = 5
w = 4
compression_algos = ['standard','srht','cwt']
results = []

for m in sizes:
    n = int(0.75*m)

    # sparse and dense matrix
    for delta,density in [(delta_dense,'dense'),(delta_sparse,'sparse')]:
        if density == 'dense':
            A,_,_ = generate_dense_matrix(m,r,delta) 
        elif density == 'sparse':
            A,_,_ = generate_sparse_matrix(m,r,delta)

        for compression in compression_algos:

            run_errors = []
            run_times = []
            for _ in range(10):
                np.random.seed(_)
                start_time = time.perf_counter()
                X, Y, errors = randomized_nmf(np.abs(A),r,rOV,w,compression=compression)

                elapsed = time.perf_counter() - start_time
                run_times.append(elapsed)
                run_errors.append(errors[-1])
            
            results.append({
                'Size': m,
                'Density': density,
                'Compression Algo':compression,
                'Avg Errors': np.mean(run_errors),
                'Avg Time': np.mean(run_times)
            })
    print(m)

100
200
300
400
500
600
700
800
900
1000


*Plots*

In [14]:
results_df = pd.DataFrame(results)
results_dense = results_df[results_df['Density'] == 'dense']
results_sparse = results_df[results_df['Density'] == 'sparse']

In [15]:
# Create subplot grid
fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=('Reconstruction Error', 'Computation Time'),
                   horizontal_spacing=0.15)

# Add traces for each metric
for algo in results_dense['Compression Algo'].unique():
    _df = results_dense[results_dense['Compression Algo'] == algo]
    
    # Reconstruction Error (left plot)
    fig.add_trace(
        go.Scatter(
            x=_df['Size'],
            y=_df['Avg Errors'],
            name=f'{algo}',
            mode='lines+markers',
            legendgroup=algo  # Group legends across subplots
        ),
        row=1, col=1
    )
    
    # Computation Time (right plot)
    fig.add_trace(
        go.Scatter(
            x=_df['Size'],
            y=_df['Avg Time'],
            name=f'{algo}',
            mode='lines+markers',
            legendgroup=algo,
            showlegend=False  # Only show legend once
        ),
        row=1, col=2
    )

# Update layout
fig.update_layout(
    title_text='Dense Matrix Compression Performance',
    height=500,
    width=1000,
    hovermode='x unified'
)

# Update axis labels
fig.update_xaxes(title_text='Matrix Size (m)', row=1, col=1)
fig.update_xaxes(title_text='Matrix Size (m)', row=1, col=2)
fig.update_yaxes(title_text='Avg Reconstruction Error', row=1, col=1)
fig.update_yaxes(title_text='Computation Time (s)', row=1, col=2)

fig.show()

In [16]:
# Create subplot grid
fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=('Reconstruction Error', 'Computation Time'),
                   horizontal_spacing=0.15)

# Add traces for each metric
for algo in results_sparse['Compression Algo'].unique():
    _df = results_sparse[results_sparse['Compression Algo'] == algo]
    
    # Reconstruction Error (left plot)
    fig.add_trace(
        go.Scatter(
            x=_df['Size'],
            y=_df['Avg Errors'],
            name=f'{algo}',
            mode='lines+markers',
            legendgroup=algo  # Group legends across subplots
        ),
        row=1, col=1
    )
    
    # Computation Time (right plot)
    fig.add_trace(
        go.Scatter(
            x=_df['Size'],
            y=_df['Avg Time'],
            name=f'{algo}',
            mode='lines+markers',
            legendgroup=algo,
            showlegend=False  # Only show legend once
        ),
        row=1, col=2
    )

# Update layout
fig.update_layout(
    title_text='Sparse Matrix Compression Performance',
    height=500,
    width=1000,
    hovermode='x unified'
)

# Update axis labels
fig.update_xaxes(title_text='Matrix Size (m)', row=1, col=1)
fig.update_xaxes(title_text='Matrix Size (m)', row=1, col=2)
fig.update_yaxes(title_text='Avg Reconstruction Error', row=1, col=1)
fig.update_yaxes(title_text='Computation Time (s)', row=1, col=2)

fig.show()